In [2]:
import numpy
import ga

"""
The y=target is to maximize this equation ASAP:
    y = w1x1+w2x2+w3x3+w4x4+w5x5+6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)
    What are the best values for the 6 weights w1 to w6?
    We are going to use the genetic algorithm for the best possible values after a number of generations.
"""

# Inputs of the equation.
equation_inputs = [4,-2,3.5,5,-11,-4.7]

# Number of the weights we are looking to optimize.
num_weights = len(equation_inputs)

"""
Genetic algorithm parameters:
    Mating pool size
    Population size
"""
sol_per_pop = 8
num_parents_mating = 4

# Defining the population size.
pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
#Creating the initial population.
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)
print(new_population)

"""
new_population[0, :] = [2.4,  0.7, 8, -2,   5,   1.1]
new_population[1, :] = [-0.4, 2.7, 5, -1,   7,   0.1]
new_population[2, :] = [-1,   2,   2, -3,   2,   0.9]
new_population[3, :] = [4,    7,   12, 6.1, 1.4, -4]
new_population[4, :] = [3.1,  4,   0,  2.4, 4.8,  0]
new_population[5, :] = [-2,   3,   -7, 6,   3,    3]
"""

best_outputs = []
num_generations = 1000
for generation in range(num_generations):
    print("Generation : ", generation)
    # Measuring the fitness of each chromosome in the population.
    fitness = ga.cal_pop_fitness(equation_inputs, new_population)
    print("Fitness")
    print(fitness)

    best_outputs.append(numpy.max(numpy.sum(new_population*equation_inputs, axis=1)))
    # The best result in the current iteration.
    print("Best result : ", numpy.max(numpy.sum(new_population*equation_inputs, axis=1)))
    
    # Selecting the best parents in the population for mating.
    parents = ga.select_mating_pool(new_population, fitness, 
                                      num_parents_mating)
    print("Parents")
    print(parents)

    # Generating next generation using crossover.
    offspring_crossover = ga.crossover(parents,
                                       offspring_size=(pop_size[0]-parents.shape[0], num_weights))
    print("Crossover")
    print(offspring_crossover)

    # Adding some variations to the offspring using mutation.
    offspring_mutation = ga.mutation(offspring_crossover, num_mutations=2)
    print("Mutation")
    print(offspring_mutation)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation
    
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = ga.cal_pop_fitness(equation_inputs, new_population)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = numpy.where(fitness == numpy.max(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])


import matplotlib.pyplot
matplotlib.pyplot.plot(best_outputs)
matplotlib.pyplot.xlabel("Iteration")
matplotlib.pyplot.ylabel("Fitness")
matplotlib.pyplot.show()

[[ 3.66470828  0.20073164  1.4364188  -3.32724429 -2.05916511  3.88823537]
 [-3.03490259  2.25282179  1.88798773 -3.42090299  2.3012063   2.36796302]
 [-1.27645822 -2.88408187 -3.82169431  1.10452568  0.19773621 -3.93262876]
 [-0.8200804   3.88351353 -0.88128093 -2.89894327  0.15937457 -2.14277386]
 [ 1.95286185  2.37851259  1.94616576 -1.67503983 -2.19278613 -2.08750235]
 [-2.85731072  2.96073712 -0.79484689 -2.59740428  2.80974017  1.14590083]
 [-1.15992298 -3.10342188 -0.36985887 -3.12484069 -2.34730916  3.20960255]
 [-3.7421205  -2.94213847 -2.00366649 -1.81513285  1.47931801 -3.28994021]]
Generation :  0
Best result :  35.46895035434496
Generation :  1
Best result :  42.26930644665543
Generation :  2
Best result :  44.299275961386954
Generation :  3
Best result :  53.42301191796933
Generation :  4
Best result :  67.86748217587291
Best solution :  [[[ 3.66470828  0.20073164  1.4364188  -1.67503983 -4.28605315
   -2.08750235]]]
Best solution fitness :  [67.86748218]


# Prediction

In [7]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model(r'C:\Users\krish\ai-power-converter\machine-learning\model_5_256.h5')

with open(r'C:\Users\krish\ai-power-converter\machine-learning\scaler_X.pkl', 'rb') as f:
    scaler_X = pickle.load(f)

# Define the input values
input_values = np.array([0.0006521052631578,0.0001194736842105,143157.89473684208])  # Replace these values with your actual input values

# Reshape the input data to ensure it is in the correct shape for normalization
input_values_reshaped = input_values.reshape(1, -1)  # Reshape to (1, 3)

# Fit the scaler to the training data and transform the new input data
input_values_scaled = scaler_X.transform(input_values_reshaped)

# Make predictions using the model
prediction = model.predict(input_values_scaled)

# Define the original output values obtained from your simulation
original_output = np.array([0.0867577616798147,1.8022857528367369,3.1017004916475472,3.1017004916475472,0.0155677936159264,1.3457323804555696])  # Replace these values with your actual output values

# Calculate the mean absolute error between original and predicted output values
mae = np.mean(np.abs(original_output - prediction))

# Calculate Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((original_output - prediction) / original_output)) * 100

# Print the original and predicted output values for comparison
print("Original output:", original_output)
print("Predicted output:", prediction)
print("Mean Absolute Error:", mae)
print(f'Mean Absolute Percentage Error (MAPE): {mape}%')

# Assign predictions to variables
Delta_IL_percent = prediction[0][0]
Delta_Vo_percent = prediction[0][1]
P_l_s1 = prediction[0][2]
P_l_s2 = prediction[0][3]
P_l_C = prediction[0][4]
P_l_L_Cu = prediction[0][5]

# Print the predictions
print("Delta Current:", Delta_IL_percent)
print("Delta Voltage:", Delta_Vo_percent)
print("Pl_s1:", Pl_s1)
print("Pl_s2:", Pl_s2)
print("Pl_C:", Pl_C)
print("Pl_L_Cu:", Pl_L_Cu)




1/1 [==============================] - 0s 280ms/step
Original output: [0.08675776 1.80228575 3.10170049 3.10170049 0.01556779 1.34573238]
Predicted output: [[0.07611273 2.215468   3.096919   3.093316   0.01535417 1.3317415 ]]
Mean Absolute Error: 0.07519960021255918
Mean Absolute Percentage Error (MAPE): 6.338601044481275%
Delta Current: 0.07611273
Delta Voltage: 2.215468
Pl_s1: 3.096919
Pl_s2: 3.093316
Pl_C: 0.015354172
Pl_L_Cu: 1.3317415


# Optimization

In [ ]:
import numpy as np
import ga
import pickle
import tensorflow as tf

# Load the machine learning model and scaler
model = tf.keras.models.load_model(r'C:\Users\krish\ai-power-converter\machine-learning\model_5_256.h5')
with open(r'C:\Users\krish\ai-power-converter\machine-learning\scaler_X.pkl', 'rb') as f:
    scaler_X = pickle.load(f)

# Define constraint values
Vol_lim = 7e-6
Delta_Vo_percent_lim = 1
Delta_IL_percent_lim = 10

# Genetic algorithm parameters
sol_per_pop = 8
num_parents_mating = 4
pop_size = (sol_per_pop, 3)  # We have 3 variables: fs, L, C

# Define the fitness function
def calculate_fitness(individual):
    # Decode the individual to get fs, L, and C values
    fs = individual[0]
    L = individual[1]
    C = individual[2]
    
    # Prepare input values for prediction
    input_values = np.array([L, C, fs])
    input_values_reshaped = input_values.reshape(1, -1)
    
    # Scale input values using the loaded scaler
    input_values_scaled = scaler_X.transform(input_values_reshaped)
    
    # Make predictions using the loaded model
    prediction = model.predict(input_values_scaled)
    
    # Extract predictions from the model output
    Delta_IL_percent = prediction[0][0]
    Delta_Vo_percent = prediction[0][1]
    P_l_s1 = prediction[0][2]
    P_l_s2 = prediction[0][3]
    P_l_C = prediction[0][4]
    P_l_L_Cu = prediction[0][5]
    
    # Calculate the objective function value based on the given equation
    objective_value = P_l_s1 + P_l_s2 + P_l_L_Cu + P_l_C
    
    return objective_value, Delta_IL_percent, Delta_Vo_percent


# Define the constraints function
def check_constraints(individual, Delta_IL_percent, Delta_Vo_percent):
    fs = individual[0]
    L = individual[1]
    C = individual[2]
    
    # Define the constraint values
    Vol_L =  # Define Vol_L based on your problem-specific requirements
    Vol_C =  # Define Vol_C based on your problem-specific requirements
    
    # Check if constraints are satisfied
    constraint1 = Vol_L + Vol_C <= Vol_lim
    constraint2 = Delta_Vo_percent <= Delta_Vo_percent_lim
    constraint3 = Delta_IL_percent <= Delta_IL_percent_lim
    
    return constraint1 and constraint2 and constraint3


# Genetic Algorithm optimization
best_outputs = []
num_generations = 1000
new_population = np.random.uniform(low=-4.0, high=4.0, size=pop_size)

for generation in range(num_generations):
    print("Generation : ", generation)
    
    # Measuring the fitness of each chromosome in the population.
    fitness_values = []
    delta_IL_percent_values = []
    delta_Vo_percent_values = []
    for individual in new_population:
        fitness_value, delta_IL_percent, delta_Vo_percent = calculate_fitness(individual)
        fitness_values.append(fitness_value)
        delta_IL_percent_values.append(delta_IL_percent)
        delta_Vo_percent_values.append(delta_Vo_percent)
    fitness = np.array(fitness_values)
    delta_IL_percent = np.array(delta_IL_percent_values)
    delta_Vo_percent = np.array(delta_Vo_percent_values)
    
    print("Fitness")
    print(fitness)

    best_outputs.append(np.min(fitness))
    print("Best result : ", np.min(fitness))
    
    # Check and enforce constraints
    for i in range(sol_per_pop):
        if not check_constraints(new_population[i], delta_IL_percent[i], delta_Vo_percent[i]):
            # Regenerate individual until it satisfies the constraints
            while not check_constraints(new_population[i], delta_IL_percent[i], delta_Vo_percent[i]):
                new_population[i] = np.random.uniform(low=-4.0, high=4.0, size=(1, 3))

    # Selecting the best parents in the population for mating.
    parents = ga.select_mating_pool(new_population, fitness, num_parents_mating)
    print("Parents")
    print(parents)

    # Generating next generation using crossover.
    offspring_crossover = ga.crossover(parents, offspring_size=(pop_size[0]-parents.shape[0], 3))
    print("Crossover")
    print(offspring_crossover)

    # Adding some variations to the offspring using mutation.
    offspring_mutation = ga.mutation(offspring_crossover, num_mutations=2)
    print("Mutation")
    print(offspring_mutation)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

# Getting the best solution after iterating finishing all generations.
# At first, the fitness is calculated for each solution in the final generation.
fitness_values = []
delta_IL_percent_values = []
delta_Vo_percent_values = []
for individual in new_population:
    fitness_value, delta_IL_percent, delta_Vo_percent = calculate_fitness(individual)
    fitness_values.append(fitness_value)
    delta_IL_percent_values.append(delta_IL_percent)
    delta_Vo_percent_values.append(delta_Vo_percent)
fitness = np.array(fitness_values)
delta_IL_percent = np.array(delta_IL_percent_values)
delta_Vo_percent = np.array(delta_Vo_percent_values)

# Then return the index of that solution corresponding to the best fitness.
best_match_idx = np.where(fitness == np.min(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])

import matplotlib.pyplot as plt
plt.plot(best_outputs)
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.show()
